# 02: Observability with Langfuse

This notebook shows how to use **Langfuse** for observing and debugging the Knowledge Agent.
Langfuse captures traces of agent execution, including tool calls, LLM interactions, and timing.

## What You'll Learn

1. **Enabling Tracing** - How to set up Langfuse tracing
2. **Running with Traces** - Execute agent queries with full observability
3. **Viewing Traces** - Use the Langfuse UI to inspect agent behavior
4. **Loading Traces via API** - Programmatically fetch and analyze traces

## Prerequisites

- Set these in your `.env` file:
  - `GOOGLE_API_KEY` - For the agent
  - `LANGFUSE_PUBLIC_KEY` - From Langfuse dashboard
  - `LANGFUSE_SECRET_KEY` - From Langfuse dashboard
  - `LANGFUSE_HOST` (optional) - Defaults to `https://cloud.langfuse.com`

In [ ]:
# Setup - all imports at the top
import os

from aieng.agent_evals.knowledge_agent import KnowledgeGroundedAgent
from aieng.agent_evals.knowledge_agent.tracing import flush_traces, init_tracing
from dotenv import load_dotenv
from langfuse import Langfuse
from rich.console import Console
from rich.panel import Panel
from rich.table import Table


load_dotenv(verbose=True)
console = Console(width=100)

# Check Langfuse configuration
langfuse_configured = all(
    [
        os.getenv("LANGFUSE_PUBLIC_KEY"),
        os.getenv("LANGFUSE_SECRET_KEY"),
    ]
)

if langfuse_configured:
    console.print("[green]✓[/green] Langfuse credentials found")
else:
    console.print("[red]✗[/red] Langfuse credentials not found")
    console.print("[dim]Set LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY in .env[/dim]")

## 1. How Tracing Works

The Knowledge Agent uses **OpenTelemetry** for tracing, which Langfuse captures automatically.

### Trace Structure

```
Trace: "agent-query-12345"
├── Span: "planning"
│   └── LLM Call: "create_plan"
├── Span: "step-1-execution"
│   ├── Tool Call: "google_search"
│   ├── Tool Call: "web_fetch"
│   └── LLM Call: "summarize_step"
├── Span: "reflection"
│   └── LLM Call: "reflect_on_findings"
├── Span: "step-2-execution"
│   └── ...
└── Span: "synthesis"
    └── LLM Call: "final_answer"
```

Each trace captures:
- **Input/Output**: Question and final answer
- **Tool Calls**: Search queries, URLs fetched, file operations
- **LLM Interactions**: Prompts and responses
- **Timing**: Duration of each operation
- **Metadata**: Plan complexity, step statuses, sources used

## 2. Enabling Tracing

To enable tracing, use the `init_tracing()` function from the tracing module.

In [ ]:
# Initialize tracing
tracing_enabled = init_tracing()

if tracing_enabled:
    console.print("[green]✓[/green] Langfuse tracing initialized")
else:
    console.print("[yellow]⚠[/yellow] Tracing not enabled (missing credentials or initialization failed)")

## 3. Running the Agent with Tracing

Now let's run a query. The trace will be automatically captured and sent to Langfuse.

In [ ]:
# Create the agent
agent = KnowledgeGroundedAgent(enable_planning=True)

console.print(f"Agent model: [cyan]{agent.model}[/cyan]")

In [ ]:
# Run a query (this will be traced)
question = "What are the key features of Python 3.12?"

console.print(Panel(question, title="Question", border_style="green"))
console.print("\n[dim]Agent is researching (trace being captured)...[/dim]\n")

response = await agent.answer_async(question)

console.print(
    Panel(
        response.text[:1000] + "..." if len(response.text) > 1000 else response.text,
        title="Answer",
        border_style="cyan",
        subtitle=f"Duration: {response.total_duration_ms / 1000:.1f}s | Tools: {len(response.tool_calls)}",
    )
)

In [ ]:
# Flush traces to ensure they're sent to Langfuse
if tracing_enabled:
    flush_traces()
    console.print("[green]✓[/green] Traces flushed to Langfuse")
    console.print("\n[dim]View your traces at: https://cloud.langfuse.com[/dim]")

## 4. Viewing Traces in Langfuse UI

Open the [Langfuse Dashboard](https://cloud.langfuse.com) to view your traces.

### What to Look For

1. **Trace Timeline**: See the sequence of operations
2. **Span Details**: Click any span to see inputs/outputs
3. **LLM Calls**: View prompts and completions
4. **Tool Calls**: See search queries and fetched URLs
5. **Errors**: Red spans indicate failures

### Useful Filters

- Filter by trace name (e.g., `knowledge-agent`)
- Filter by time range
- Filter by tags (if added)
- Search by input text

## 5. Loading Traces via API

You can programmatically fetch traces using the Langfuse Python SDK.

In [ ]:
# Initialize Langfuse client
if langfuse_configured:
    langfuse = Langfuse()
    console.print("[green]✓[/green] Langfuse client initialized")
else:
    langfuse = None
    console.print("[yellow]⚠[/yellow] Langfuse client not available (missing credentials)")

In [ ]:
# Fetch recent traces
if langfuse:
    # Get traces from the last 24 hours
    traces = langfuse.fetch_traces(
        limit=10,
        order_by="timestamp",
    )

    if traces.data:
        traces_table = Table(title="Recent Traces")
        traces_table.add_column("ID", style="dim", width=20)
        traces_table.add_column("Name", style="cyan")
        traces_table.add_column("Input", style="white", width=40)
        traces_table.add_column("Duration", style="green", justify="right")

        for trace in traces.data[:10]:
            input_text = (
                str(trace.input)[:40] + "..." if trace.input and len(str(trace.input)) > 40 else str(trace.input or "")
            )
            duration = f"{trace.latency:.1f}s" if trace.latency else "N/A"
            traces_table.add_row(trace.id[:20], trace.name or "unnamed", input_text, duration)

        console.print(traces_table)
    else:
        console.print("[dim]No traces found[/dim]")
else:
    console.print("[dim]Skipping (Langfuse not configured)[/dim]")

In [ ]:
# Get details for a specific trace
if langfuse and traces.data:
    # Get the most recent trace
    trace_id = traces.data[0].id

    # Fetch full trace with observations (spans)
    trace = langfuse.fetch_trace(trace_id)

    console.print(
        Panel(
            f"[bold]Trace ID:[/bold] {trace.id}\n"
            f"[bold]Name:[/bold] {trace.name}\n"
            f"[bold]Timestamp:[/bold] {trace.timestamp}\n"
            f"[bold]Duration:[/bold] {trace.latency:.2f}s\n"
            f"[bold]Input:[/bold] {str(trace.input)[:200]}..."
            if trace.input
            else "",
            title="Trace Details",
            border_style="blue",
        )
    )

    # Show observations (spans) in the trace
    observations = langfuse.fetch_observations(trace_id=trace_id)

    if observations.data:
        obs_table = Table(title="Observations (Spans)")
        obs_table.add_column("Type", style="cyan")
        obs_table.add_column("Name", style="white")
        obs_table.add_column("Model", style="dim")
        obs_table.add_column("Duration", style="green", justify="right")

        for obs in observations.data[:15]:
            duration = f"{obs.latency:.2f}s" if obs.latency else "N/A"
            obs_table.add_row(obs.type, obs.name or "unnamed", obs.model or "-", duration)

        if len(observations.data) > 15:
            obs_table.add_row("...", f"({len(observations.data) - 15} more)", "", "")

        console.print(obs_table)
else:
    console.print("[dim]Skipping (no traces available)[/dim]")

## 6. Analyzing Trace Data

You can extract useful metrics from traces for analysis.

In [ ]:
# Analyze traces
if langfuse and traces.data:
    # Calculate aggregate metrics
    total_traces = len(traces.data)
    traces_with_latency = [t for t in traces.data if t.latency]

    if traces_with_latency:
        avg_latency = sum(t.latency for t in traces_with_latency) / len(traces_with_latency)
        max_latency = max(t.latency for t in traces_with_latency)
        min_latency = min(t.latency for t in traces_with_latency)

        metrics_table = Table(title="Trace Metrics")
        metrics_table.add_column("Metric", style="cyan")
        metrics_table.add_column("Value", style="white")

        metrics_table.add_row("Total Traces", str(total_traces))
        metrics_table.add_row("Avg Latency", f"{avg_latency:.2f}s")
        metrics_table.add_row("Max Latency", f"{max_latency:.2f}s")
        metrics_table.add_row("Min Latency", f"{min_latency:.2f}s")

        console.print(metrics_table)
    else:
        console.print("[dim]No latency data available[/dim]")
else:
    console.print("[dim]Skipping (no traces available)[/dim]")

## 7. Tracing via CLI

You can enable tracing from the command line using the `--log-trace` flag:

```bash
# Ask with tracing enabled
knowledge-agent ask "What is quantum computing?" --log-trace

# Run evaluation with tracing
knowledge-agent eval --samples 3 --log-trace
```

Traces will be sent to Langfuse and you'll see a confirmation message.

## 8. Best Practices

### When to Use Tracing

- **Development**: Debug agent behavior and tool usage
- **Testing**: Compare different configurations
- **Production**: Monitor latency and errors
- **Evaluation**: Analyze failed evaluations

### What to Look For

1. **Long-running spans**: Which steps take the most time?
2. **Tool call patterns**: Is the agent making unnecessary calls?
3. **LLM token usage**: Are prompts too long?
4. **Errors**: Why did a step fail?
5. **Plan changes**: Did the agent update its plan during execution?

### Tips

- Use descriptive trace names for easy filtering
- Add tags for categorization (e.g., "evaluation", "production")
- Set up alerts for high latency or error rates

## Summary

In this notebook, you learned:

1. **Trace Structure** - How agent execution is captured
2. **Enabling Tracing** - Using `init_tracing()` and `flush_traces()`
3. **Viewing Traces** - Using the Langfuse dashboard
4. **API Access** - Programmatically fetching and analyzing traces

Langfuse provides powerful observability for understanding and debugging your agent's behavior.

In [ ]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Resources:[/cyan]\n"
        "• Langfuse Dashboard: https://cloud.langfuse.com\n"
        "• Langfuse Docs: https://langfuse.com/docs\n"
        "• OpenTelemetry: https://opentelemetry.io",
        title="Done",
        border_style="green",
    )
)